In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os, sys

project_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_path not in sys.path:
    sys.path.append(project_path)
from utils.transformations import *

#DimUser

#Autoloader

In [0]:
df_user=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimUser/checkpoint')\
    .load('abfss://bronze@hriteshdatastorage.dfs.core.windows.net/DimUser')

In [0]:
df_user.display()

In [0]:
df_user=df_user.withColumn('user_name', upper(col('user_name')))


In [0]:
df_user.display()

In [0]:
df_user_obj=reusable()
df_user=df_user_obj.dropColumns(df_user, ['_rescued_data'])
df_user=df_user.dropDuplicates(['user_id'])

In [0]:
df_user.display()

In [0]:
df_user.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimUser/checkpoint')\
    .trigger(once=True)\
    .option('path','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimUser/data')\
.toTable('spotify_cata.silver.DimUser')

#DimArtist

In [0]:
df_artist=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimArtist/checkpoint')\
    .load('abfss://bronze@hriteshdatastorage.dfs.core.windows.net/DimArtist')

In [0]:
df_artist.display()

In [0]:
df_art_obj=reusable()
df_artist=df_art_obj.dropColumns(df_artist, ['_rescued_data'])
df_artist=df_artist.dropDuplicates(['artist_id'])

In [0]:
df_artist.display()

In [0]:
df_artist.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimArtist/checkpoint')\
    .trigger(once=True)\
    .option('path','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimArtist/data')\
    .toTable('spotify_cata.silver.DimArtist')

#DimTrack

In [0]:
df_track=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimTrack/checkpoint')\
    .load('abfss://bronze@hriteshdatastorage.dfs.core.windows.net/DimTrack')

In [0]:
df_track.display()

In [0]:
df_track=df_track.withColumn('duaration_flag',when(col('duration_sec')<150,'Short').when(col("duration_sec")<300,'Medium').otherwise('Long'))
df_track=df_track.withColumn('track_name',regexp_replace(col('track_name'),'-',' '))
df_track=reusable().dropColumns(df_track,['_rescued_data'])
df_track=df_track.dropDuplicates(['track_id'])

In [0]:
df_track.display()

In [0]:
df_track.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimTrack/checkpoint')\
    .trigger(once=True)\
    .option('path','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimTrack/data')\
    .toTable('spotify_cata.silver.DimTrack')

#DimDate

In [0]:
df_date=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimDate/checkpoint')\
    .load('abfss://bronze@hriteshdatastorage.dfs.core.windows.net/DimDate')

In [0]:
df_date.display()

In [0]:
df_date=reusable().dropColumns(df_date,['_rescued_data'])
df_date.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimDate/checkpoint')\
    .trigger(once=True)\
    .option('path','abfss://silver@hriteshdatastorage.dfs.core.windows.net/DimDate/data')\
    .toTable('spotify_cata.silver.DimDate')

#FactStream

In [0]:
df_fact_stream=spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/FactStream/checkpoint')\
    .load('abfss://bronze@hriteshdatastorage.dfs.core.windows.net/FactStream')
df_fact_stream=reusable().dropColumns(df_fact_stream,['_rescued_data'])

In [0]:
df_fact_stream.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation','abfss://silver@hriteshdatastorage.dfs.core.windows.net/FactStream/checkpoint')\
    .trigger(once=True)\
    .option('path','abfss://silver@hriteshdatastorage.dfs.core.windows.net/FactStream/data')\
    .toTable('spotify_cata.silver.FactStream')